In [1]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter

In [2]:
df_price = pd.read_parquet("../../technical_indicators/btcusd_hourly_features.parquet")
df_news = pd.read_parquet("../../crypto_news/data/6b.cryptonews_topic.parquet")
df_reddit = pd.read_parquet("../../reddit/reddit_sentiment_processed.parquet.gzip")

### preprocess df_news

In [ ]:
# remove original text
df_news.drop(columns=['title', 'text'], inplace=True)

# format date
df_news['date'] = pd.to_datetime(df_news['date'], utc=True)

# one-hot encode the news source
top_sources = df_news['source_name'].value_counts().nlargest(15).index
df_news['source_name'] = df_news['source_name'].apply(lambda x: x if x in top_sources else 'Other')
df_news = pd.get_dummies(df_news, columns=['source_name'], prefix='source')

# preprocess CryptoBERT sentiment
conditions = [
  (df_news['sentiment_CryptoBERT'] == "LABEL_1") & (df_news['score_CryptoBERT'] > 0.9),
  (df_news['sentiment_CryptoBERT'] == "LABEL_0") & (df_news['score_CryptoBERT'] > 0.9)
]
df_news['sentiment_CryptoBERT'] = np.select(conditions, [1, -1], default=0)

# preprocess FinBERT sentiment
mask_positive = df_news['sentiment_finbert'] == "positive"
df_news.loc[mask_positive, 'sentiment_finbert'] = np.where(
  df_news.loc[mask_positive, 'score_finbert'] > 0.75, 1, 0
)
mask_negative = df_news['sentiment_finbert'] == "negative"
df_news.loc[mask_negative, 'sentiment_finbert'] = np.where(
  df_news.loc[mask_negative, 'score_finbert'] > 0.75, -1, 0
)
mask_neutral = df_news['sentiment_finbert'] == "neutral"
df_news.loc[mask_neutral, 'sentiment_finbert'] = 0

df_news.drop(columns=['score_CryptoBERT', 'score_finbert'], inplace=True)

# preprocess topic
df_news['topic'] = np.where(df_news['topic_confidence_score'] < 0.4, 'others', df_news['topic'])
top_topics = df_news['topic'].value_counts().nlargest(5).index
df_news['topic'] = df_news['topic'].apply(lambda x: x if x in top_topics else 'Other')
df_news = pd.get_dummies(df_news, columns=['topic'], prefix='topic')
df_news.drop(columns=['topic_confidence_score'], inplace=True)

# preprocess NER
stopword_set = set(stopwords.words('english'))
df_news['NER'] = df_news['NER'].apply(
  lambda entities: [entity[0] for entity in entities]
  if isinstance(entities, (list, np.ndarray)) else list()
)
df_news['NER'] = df_news['NER'].apply(
  lambda ner_list: [re.sub(r'[^\w\s]', '', entity).lower() for entity in ner_list]
)
df_news['NER'] = df_news['NER'].apply(
  lambda ner_list: [entity for entity in ner_list if len(entity) > 1 and entity not in stopword_set and not entity in ["first", "one", "daily", "today", "2024", "two", "this week"]]
)

all_tokens = [token for tokens_list in df_news['NER'] for token in tokens_list]
top_entities = Counter(all_tokens).most_common(10)

top_entities_tokens = [entity for entity, _ in top_entities]
df_news['NER'] = df_news['NER'].apply(lambda tokens: [t for t in tokens if t in top_entities_tokens])

mlb = MultiLabelBinarizer()
ner_encoded = pd.DataFrame(
  mlb.fit_transform(df_news['NER']),
  columns=[f'NER_{ent}' for ent in mlb.classes_],
  index=df_news.index
)
df_news = pd.concat([df_news, ner_encoded], axis=1)

print(df_news.shape)
df_news.head()

(159486, 42)


,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa,NER,sentiment_CryptoBERT,sentiment_finbert,...,NER_binance,NER_blackrock,NER_coinbase,NER_el salvador,NER_fed,NER_grayscale,NER_michael saylor,NER_microstrategy,NER_sec,NER_us
0,2021-01-01 16:07:58+00:00,0.0,0.0,0.3,0.0,0.7,0.0,[sec],1,0,...,0,0,0,0,0,0,0,0,1,0
1,2021-01-01 14:51:10+00:00,0.9,0.0,0.0,0.0,0.5,0.0,[],1,1,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-03 23:46:20+00:00,0.8,0.0,0.0,0.0,0.0,0.0,[],1,1,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-05 01:29:35+00:00,0.8,0.0,0.0,0.0,0.0,0.0,[],1,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-04 14:01:49+00:00,-0.8,0.0,0.0,0.0,0.0,0.0,[],-1,-1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# group 

aggregation_dict = {
  'date': 'count',  # for counting the number of rows
  'tanalysis_absa': 'mean',
  'economy_absa': 'mean',
  'regulation_absa': 'mean',
  'technology_absa': 'mean',
  'adoption_absa': 'mean',
  'cybersecurity_absa': 'mean',
  'source_AMBCrypto': 'sum',
  'source_BeInCrypto': 'sum',
  'source_Benzinga': 'sum',
  'source_Bitcoin': 'sum',
  'source_Bitcoin Magazine': 'sum',
  'source_CoinPedia': 'sum',
  'source_Coindesk': 'sum',
  'source_Coingape': 'sum',
  'source_Cointelegraph': 'sum',
  'source_CryptoPotato': 'sum',
  'source_Cryptopolitan': 'sum',
  'source_NewsBTC': 'sum',
  'source_Other': 'sum',
  'source_The Currency Analytics': 'sum',
  'source_The Daily Hodl': 'sum',
  'source_UToday': 'sum',
}

bool_cols = [
  'topic_Other',
  'topic_exchange traded funds, ETF',
  'topic_institutional investments',
  'topic_market sentiment',
  'topic_others',
  'topic_price action, price movement, trading',
  'NER_binance',
  'NER_blackrock',
  'NER_coinbase',
  'NER_el salvador',
  'NER_fed',
  'NER_grayscale',
  'NER_michael saylor',
  'NER_microstrategy',
  'NER_sec',
  'NER_us'
]

bool_agg = {col: lambda x: int(x.eq(1).any()) for col in bool_cols}
combined_agg = {**aggregation_dict, **bool_agg}

df_news = df_news.groupby(pd.Grouper(key='date', freq='h')).agg(combined_agg)
df_news.rename(columns={'date': 'news_count'}, inplace=True)

# set index to date
df_news.reset_index(inplace=True)
df_news.set_index('date', inplace=True)

# timestamp formatting
# df_news.index = df_news.index.tz_localize(None)

df_news

,news_count,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa,source_AMBCrypto,source_BeInCrypto,source_Benzinga,...,NER_binance,NER_blackrock,NER_coinbase,NER_el salvador,NER_fed,NER_grayscale,NER_michael saylor,NER_microstrategy,NER_sec,NER_us
date,,,,,,,,,,,,,,,,,,,,,
2021-01-01 06:00:00+00:00,2,0.850000,0.000,0.00,0.0,0.150,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2021-01-01 07:00:00+00:00,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-01-01 08:00:00+00:00,2,0.400000,0.000,0.00,0.0,0.500,0.0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
2021-01-01 09:00:00+00:00,2,-0.050000,0.000,0.00,0.0,0.400,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2021-01-01 10:00:00+00:00,3,0.766667,0.000,0.00,0.0,0.300,0.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 00:00:00+00:00,4,0.025000,0.175,0.30,0.0,0.325,0.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-01-01 01:00:00+00:00,1,0.800000,0.000,0.00,0.0,0.000,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-01-01 02:00:00+00:00,2,0.050000,0.000,0.00,0.0,0.250,0.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
# timestamp formatting
df_news.index = df_news.index.tz_localize(None)
df_news

,news_count,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa,source_AMBCrypto,source_BeInCrypto,source_Benzinga,...,NER_binance,NER_blackrock,NER_coinbase,NER_el salvador,NER_fed,NER_grayscale,NER_michael saylor,NER_microstrategy,NER_sec,NER_us
date,,,,,,,,,,,,,,,,,,,,,
2021-01-01 06:00:00,2,0.850000,0.000,0.00,0.0,0.150,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2021-01-01 07:00:00,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-01-01 08:00:00,2,0.400000,0.000,0.00,0.0,0.500,0.0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
2021-01-01 09:00:00,2,-0.050000,0.000,0.00,0.0,0.400,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2021-01-01 10:00:00,3,0.766667,0.000,0.00,0.0,0.300,0.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 00:00:00,4,0.025000,0.175,0.30,0.0,0.325,0.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-01-01 01:00:00,1,0.800000,0.000,0.00,0.0,0.000,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-01-01 02:00:00,2,0.050000,0.000,0.00,0.0,0.250,0.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


### df_price

In [6]:
df_price

,return_forward,open,high,low,close,volume,return,sma_5,sma_20,sma_50,...,upper_band_10,middle_band_10,lower_band_10,upper_band_20,middle_band_20,lower_band_20,10_band_width,pct_b_10,20_band_width,pct_b_20
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,0.008612,28912.47,29052.03,28776.17,28949.03,365.064315,0.000970,28736.930,28854.8905,28299.9662,...,29223.387286,28741.153,28258.918714,29307.198761,28854.8905,28402.582239,0.033557,0.715535,0.031351,0.604066
2021-01-01 01:00:00,-0.007061,28949.71,29200.00,28921.64,29198.34,520.772208,0.008612,28813.352,28858.0050,28352.3742,...,29240.874492,28744.982,28249.089508,29318.792740,28858.0050,28397.217260,0.034503,0.957113,0.031935,0.869297
2021-01-01 02:00:00,0.005256,29200.00,29219.01,28947.70,28992.18,394.441884,-0.007061,28932.070,28869.1935,28394.1632,...,29256.754515,28750.826,28244.897485,29331.599971,28869.1935,28406.787029,0.035194,0.738526,0.032035,0.632985
2021-01-01 03:00:00,-0.005207,28987.60,29191.60,28945.26,29144.56,220.722444,0.005256,29041.018,28889.6280,28437.3836,...,29351.781840,28798.405,28245.028160,29362.660536,28889.6280,28416.595464,0.038431,0.812766,0.032748,0.769466
2021-01-01 04:00:00,0.002213,29135.25,29161.46,28843.74,28992.79,500.353472,-0.005207,29055.380,28887.0185,28479.2488,...,29380.790455,28842.084,28303.377545,29357.161215,28887.0185,28416.875785,0.037356,0.639878,0.032550,0.612489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,-0.006308,93760.00,94158.00,93600.00,94008.00,7.863550,0.002666,94342.200,94002.9000,93690.0400,...,95801.067876,94516.700,93232.332124,95433.425204,94002.9000,92572.374796,0.027178,0.301965,0.030436,0.501783
2024-12-31 20:00:00,0.002858,94025.00,94059.00,93300.00,93415.00,4.565475,-0.006308,93968.200,93998.4000,93688.5000,...,95876.395307,94461.600,93046.804693,95435.711991,93998.4000,92561.088009,0.029955,0.130123,0.030582,0.297052
2024-12-31 21:00:00,-0.004142,93386.00,93777.00,93336.00,93682.00,28.484950,0.002858,93768.800,94007.2500,93693.5200,...,95898.271810,94421.000,92943.728190,95434.444994,94007.2500,92580.055006,0.031291,0.249877,0.030364,0.386053


### preprocess df_reddit

In [7]:
df_reddit

,date,total_sentiment,average_sentiment
0,2021-01-01 00:00:00,0.2966,0.015611
1,2021-01-01 01:00:00,1.6673,0.049038
2,2021-01-01 02:00:00,0.4588,0.021848
3,2021-01-01 03:00:00,-0.4329,-0.020614
4,2021-01-01 04:00:00,0.0000,0.000000
...,...,...,...
34857,2024-12-31 19:00:00,0.0000,0.000000
34858,2024-12-31 20:00:00,0.0000,0.000000
34859,2024-12-31 21:00:00,0.0000,0.000000
34860,2024-12-31 22:00:00,0.6312,0.028691


In [8]:
df_reddit['date'] = pd.to_datetime(df_reddit['date'])

df_reddit.rename(columns={'total_sentiment': 'reddit_total_sentiment',
                          'average_sentiment': 'reddit_average_sentiment'
                          }, inplace=True)

df_reddit.set_index('date', inplace=True)
df_reddit

,reddit_total_sentiment,reddit_average_sentiment
date,,
2021-01-01 00:00:00,0.2966,0.015611
2021-01-01 01:00:00,1.6673,0.049038
2021-01-01 02:00:00,0.4588,0.021848
2021-01-01 03:00:00,-0.4329,-0.020614
2021-01-01 04:00:00,0.0000,0.000000
...,...,...
2024-12-31 19:00:00,0.0000,0.000000
2024-12-31 20:00:00,0.0000,0.000000
2024-12-31 21:00:00,0.0000,0.000000


### Merge all dataframe

In [9]:
# merge df_news and df_price and df_reddit

df = df_news.merge(df_price, left_index=True, right_index=True, how='outer')
df = df.merge(df_reddit, left_index=True, right_index=True, how='outer')
df = df.iloc[6:]

In [10]:
df

,news_count,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa,source_AMBCrypto,source_BeInCrypto,source_Benzinga,...,lower_band_10,upper_band_20,middle_band_20,lower_band_20,10_band_width,pct_b_10,20_band_width,pct_b_20,reddit_total_sentiment,reddit_average_sentiment
2021-01-01 06:00:00,2.0,0.850000,0.000,0.00,0.0,0.150,0.0,0.0,0.0,0.0,...,28401.688294,29450.949582,28913.7180,28376.486418,0.039453,0.942558,0.037161,1.026218,-0.891,-0.111375
2021-01-01 07:00:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,28615.454707,29488.063359,28931.4545,28374.845641,0.030480,0.715670,0.038478,0.785547,0.000,0.000000
2021-01-01 08:00:00,2.0,0.400000,0.000,0.00,0.0,0.500,0.0,0.0,1.0,0.0,...,28782.451560,29515.716093,28939.8210,28363.925907,0.024058,0.803534,0.039799,0.852329,0.000,0.000000
2021-01-01 09:00:00,2.0,-0.050000,0.000,0.00,0.0,0.400,0.0,0.0,0.0,0.0,...,28839.212780,29553.465096,28957.6910,28361.916904,0.022668,0.682311,0.041148,0.779207,0.000,0.000000
2021-01-01 10:00:00,3.0,0.766667,0.000,0.00,0.0,0.300,0.0,1.0,0.0,0.0,...,28901.714287,29578.630359,28970.1450,28361.659641,0.020372,0.572225,0.042008,0.723469,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 00:00:00,4.0,0.025000,0.175,0.30,0.0,0.325,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-01 01:00:00,1.0,0.800000,0.000,0.00,0.0,0.000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-01 02:00:00,2.0,0.050000,0.000,0.00,0.0,0.250,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-01-01 03:00:00,2.0,0.500000,-0.100,0.00,0.0,0.000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.to_parquet("final_merged_dataset.parquet", index=True)